In [1]:
import numpy as np
import time
from skimage.morphology import label
import requests
from bs4 import BeautifulSoup

### Puzzle details
<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2019/03/20190306_20_4_7.png" width="500" height="600">

In [7]:
grid2 =np.array([[0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0],
                 [0,0,0,0,0,0,0]])

top_looks2 = [6,4,4,6,7,3,3]
bot_looks2 = [2,6,7,7,2,4,7]
left_looks2 = [4,6,5,7,5,2,7]
right_looks2 = [2,6,7,7,2,4,7]

grid1=np.array([[0,7,6,0,0,0,0],
                [0,0,0,6,6,0,0],
                [5,0,0,0,0,0,0],
                [0,6,0,0,0,4,0],
                [0,0,0,0,0,0,6],
                [0,0,4,7,0,0,0],
                [0,0,0,0,7,7,5]])

top_looks1 = [0,0,0,0,0,5,7]
bot_looks1 = [7,2,0,0,0,0,0]
left_looks1 = [0,3,0,0,5,0,1]
right_looks1 = [7,2,0,0,0,0,0]

grid1[grid1==0]=-1
grid2[grid2==0]=-1

In [3]:
#Forming the Puzzle Grid
class Matrix():
    def __init__(self,input_grid,top_looks,bot_looks,left_looks,right_looks):
        self.grid = input_grid
        self.top_looks = top_looks
        self.bot_looks = bot_looks
        self.left_looks =left_looks
        self.right_looks = right_looks
        self.counts = {i:i-np.sum(self.grid==i) for i in range(8)}
        print(self.counts)
        print('The problem - Start {}'.format(time.strftime('%X')))
        self.printGrid()
    
#Function to print the grid
    def printGrid(self):
        print(self.grid)
        
#Function to check if a digit can be placed in the given block
#Various checks
    def possible(self,row,col,digit):

        # Check row and columns count
        if digit ==0:
            if np.sum(self.grid[:,col]==0) > 2:
                #print('fail1')
                return False
            if np.sum(self.grid[row,:]==0) > 2:
                #print('fail2')
                return False
            
        # Check that final number completes
            if np.sum(self.grid[row,:]==-1) ==1:
                 if np.sum(self.grid[row,:])+digit+1  != 20:
                        #print('fail3')
                        return False
                    
            if np.sum(self.grid[:,col]==-1) ==1:
                 if np.sum(self.grid[:,col])+digit+1  != 20:
                        #print('fail4')
                        return False
        
        # Check row and columns sum
        if np.sum(self.grid[:,col])+digit+np.sum(self.grid[:,col]==-1) > 20:
            #print('fail5')
            #print(col,self.grid[:,col],np.sum(self.grid[:,col]),digit,np.sum(self.grid[:,col]==-1))
            return False
        
        if np.sum(self.grid[row,:])+digit+np.sum(self.grid[row,:]==-1)  > 20:
            #print('fail6')
            return False
        
        #check 2x2
        if digit !=0:
            if self.twobytwo(row,col):
                #print('fail7')
                return False
            
        # Check Fobidden nums
            if self.more_constraints(self.grid[row,:],digit):
                #print('fail8')
                return False
        
            if self.more_constraints(self.grid[:,col],digit):
                #print('fail9')
                return False
            
            if self.check_looks(row,col,digit):
                return False
        
        #checkconnected
        if np.max(label(self.grid!=0,connectivity=1)) >1 :
            #print('fail10')
            return False
        
        # All tests pass return True
        return True
    
    def twobytwo(self,i,j):
        if (i > 0 and j > 0 and 
        self.grid[i-1,j-1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True
    
        if (i > 0 and j < 6 and
        self.grid[i-1,j+1] > 0 and 
        self.grid[i-1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        if (i < 6 and j > 0 and
        self.grid[i+1,j-1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j-1] > 0):
            return True

    
        if (i < 6 and j < 6 and
        self.grid[i+1,j+1] > 0 and 
        self.grid[i+1,j] > 0 and 
        self.grid[i,j+1] > 0):
            return True
    
        return False
    
    def more_constraints(self,array,digit):
    
        if np.sum(array==1) > 0:
            if digit < 5:
                return True
            return False
        
        if np.sum(array==2) > 0:
            if digit < 4 :
                return True
            return False
    
        if np.sum(array==3) > 0:
            if digit < 3 :
                return True
            return False
    
        if np.sum(array==4) > 0:
            if digit < 2 :
                return True
            return False
        
        return False
        
    def check_looks(self,row,col,digit):
        if self.left_looks[row] > 0:
            found = False
            for i in range(5):
                if found == False:
                    if self.grid[row,i] != 0:
                        found = True
                        if i == col:
                            if self.left_looks[row] != digit:
                                return True
    
        if self.top_looks[col] > 0:
            found = False
            for i in range(5):
                if found == False:
                    if self.grid[i,col] != 0:
                        found = True
                        if i == row:
                            if self.top_looks[col] != digit:
                                return True

        if self.right_looks[row] > 0:
            found = False
            for i in range(6,4,-1):
                if found == False:
                    if self.grid[row,i] != 0:
                        found = True
                        if i == col:
                            if self.right_looks[row] != digit:
                                return True
    
        if self.bot_looks[col] > 0:
            found = False
            for i in range(6,4,-1):
                if found == False:
                    if self.grid[i,col] != 0:
                        found = True
                        if i == row:
                            if self.bot_looks[col] != digit:
                                return True
        return False    
        
        
    def solve(self):
        
        for row in range(7):
            for col in range(7):
                if self.grid[row,col] == -1:
                    for digit in range(7,-1,-1):
                        if (digit ==0 or self.counts[digit]>0):
                            if self.possible(row,col,digit):
                                #print(row,col,digit,self.grid[row,col],self.grid[0,0])
                                #print(self.counts)
                                #print(self.grid)
                                self.grid[row,col] = digit
                                self.counts[digit] -= 1
                                self.solve()
                                self.grid[row,col] = -1  #Backtrack step
                                self.counts[digit] += 1
                    return 
        
        print('\nThe solution - End {}'.format(time.strftime('%X')))
        self.printGrid()

In [4]:
x = Matrix(grid1,top_looks1,bot_looks1,left_looks1,right_looks1)
x.solve()
print('\n Finish {}'.format(time.strftime('%X')))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 2, 5: 3, 6: 1, 7: 3}
The problem - Start 13:42:30
[[-1  7  6 -1 -1 -1 -1]
 [-1 -1 -1  6  6 -1 -1]
 [ 5 -1 -1 -1 -1 -1 -1]
 [-1  6 -1 -1 -1  4 -1]
 [-1 -1 -1 -1 -1 -1  6]
 [-1 -1  4  7 -1 -1 -1]
 [-1 -1 -1 -1  7  7  5]]

The solution - End 13:42:58
[[3 7 6 0 4 0 0]
 [0 0 3 6 6 5 0]
 [5 5 0 6 0 4 0]
 [0 6 0 0 3 4 7]
 [5 2 7 0 0 0 6]
 [7 0 4 7 0 0 2]
 [0 0 0 1 7 7 5]]

 Finish 13:43:05


In [ ]:
x = Matrix(grid2,top_looks2,bot_looks2,left_looks2,right_looks2)
x.solve()
print('\n Finish {}'.format(time.strftime('%X')))

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7}
The problem - Start 13:44:47
[[-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1]]


### Solution is :

<img src="https://www.janestreet.com/puzzles/wp-content/uploads/2019/04/20190329_20_4_7_solved.png" width="500" height="600">

In [6]:
grid2_solve =np.array([[3,7,6,0,4,0,0],
                      [0,0,3,6,6,5,0],
                      [5,5,0,6,0,4,0],
                      [0,6,0,0,3,4,7],
                      [5,2,7,0,0,0,6],
                      [7,0,4,7,0,0,2],
                      [0,0,0,1,7,7,5]])